In [1]:
%%capture
%pip install unsloth

### Load Model and tokenizer

In [2]:
from transformers import TextStreamer
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
)

<ipython-input-2-877a0d942937>:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

### Enable the Unsloth fast inference

In [3]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

### PROMPT

In [4]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
    [
        prompt_style.format(
            "You are a professional machine learning engineer",
            "How would you deal with NaN validation loss?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a professional machine learning engineer

### Input:
How would you deal with NaN validation loss?

### Response:
I would first try to understand why the validation loss is NaN. One common reason for NaN validation loss is that the model is underfitting the training data. In this case, I would try to increase the complexity of the model or collect more training data. Another reason for NaN validation loss is that the model is overfitting the training data. In this case, I would try to reduce the complexity of the model or collect more validation data. If the model is not underfitting or overfitting, then I would check for other potential causes of NaN validation loss, such as numerical instability or incorrect hyperparameter values.
<|end_of_text|>


### Fine-tuning Llama 3.1 model using Unsloth and lighteval/MATH dataset

In [6]:
from huggingface_hub import login
from google.colab import userdata
login(userdata.get('HF_TOKEN'))

In [7]:
import wandb

wb_token = userdata.get("WB_TOKEN")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama-3.1-8B-bnb-4bit on Math Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: munumaqim-jutulivar (munumaqim-jutulivar-swerp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a security analyst who can give kali linux commands to do penetration testing. Please answer the following penetration testing question.

### Input:
{}

### Response:
{}"""

In [10]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["instruction"]
    outputs      = examples["response"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [11]:
from datasets import load_dataset

dataset = load_dataset("suryanshp1/kali-linux-pentesting-data", split="train[0:500]", trust_remote_code=True)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][0]

README.md:   0%|          | 0.00/314 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/214k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/343 [00:00<?, ? examples/s]

Map:   0%|          | 0/343 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a security analyst who can give kali linux commands to do penetration testing. Please answer the following penetration testing question.\n\n### Input:\nScan my network for any open ports\n\n### Response:\nTo scan your network for open ports, you can use the Nmap tool, which is a powerful network scanning utility available in Kali Linux. Here's a basic command to get you started:\n\n```\nnmap -sT <target_IP_address>\n```\n\nReplace `<target_IP_address>` with the IP address of the system or network you want to scan. The `-sT` option performs a TCP connect scan, which is a basic scan type that works by attempting to establish a full TCP connection with the target host.\n\nIf you want to scan a range of IP addresses, you can specify the range like this:\n\n```\nnmap -sT 192.168.1.1-100\n```\n\nThis comman

### Setting up the model

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",

    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/343 [00:00<?, ? examples/s]

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 343 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.267600
2,1.380400
3,1.243900
4,1.200400
5,1.255800
6,1.078500
7,0.949300
8,0.902600
9,0.900700
10,0.776700


In [19]:
import torch
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
gpu_stats = torch.cuda.get_device_properties(0)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

GPU = Tesla T4. Max memory = 14.741 GB.
12.883 GB of memory reserved.
758.8437 seconds used for training.
12.65 minutes used for training.
Peak reserved memory = 12.883 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 87.396 %.
Peak reserved memory for training % of max memory = 0.0 %.


### Testing model after fine-tuning

In [20]:
from IPython.display import display, Markdown

FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        prompt_style.format(
            "How to get all open ports in a system ?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=250,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
Markdown(response[0].split("\n\n### Response:")[1])


To get a list of all open ports in a system using Kali Linux, you can use the `netstat` command with the `-tulpn` options. Here's how you can do it:

1. Open a terminal in Kali Linux.
2. Type the following command to list all open ports:
```
netstat -tulpn
```
This command will display a list of all open TCP and UDP ports, along with the process ID (PID) and the name of the process that is using the port.

Alternatively, you can also use the `ss` command, which is a modern replacement for `netstat`:
```
ss -tulpn
```
This command will also display a list of all open TCP and UDP ports, along with the PID and process name.

You can also use the `nmap` command to scan the system and display a list of open ports:
```
nmap -sT -O <system_IP_address>
```
Replace `<system_IP_address>` with the IP address of the system you want to scan.

Note: Make sure to run these commands with elevated privileges (e.g., using `sudo`) if you need to scan ports that are not accessible

### Saving model in local

In [21]:
new_model_online = "suryanshp1/Llama-3.1-8B-kali-pentester"
new_model_local = "Llama-3.1-8B-kali-pentester"
model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local) # Local saving

('Llama-3.1-8B-kali-pentester/tokenizer_config.json',
 'Llama-3.1-8B-kali-pentester/special_tokens_map.json',
 'Llama-3.1-8B-kali-pentester/tokenizer.json')

### Push model online

In [22]:
model.push_to_hub(new_model_online) # Online saving
tokenizer.push_to_hub(new_model_online) # Online saving

README.md:   0%|          | 0.00/591 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/suryanshp1/Llama-3.1-8B-kali-pentester


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

### Merging and Exporting the Model - Extended (Kernel Interrupted)

In [2]:
from unsloth import FastLanguageModel

new_model_name = "suryanshp1/Llama-3.1-8B-kali-pentester"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    device_map="auto"
)
FastLanguageModel.for_inference(model);

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
from huggingface_hub import login
from google.colab import userdata
login(userdata.get('HF_TOKEN'))

In [4]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a math genius who can solve any level of algebraic problem. Please answer the following math question.

### Input:
{}

### Response:
{}"""

In [5]:
from IPython.display import display, Markdown

inputs = tokenizer(
    [
        prompt_style.format(
            "How to get operating system detail ?.",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=250,
    pad_token_id=tokenizer.eos_token_id,
)
response = tokenizer.batch_decode(outputs)
Markdown(response[0].split("\n\n### Response:")[1])



To get detailed information about the operating system, you can use the `systeminfo` command in Windows or `uname -a` in Linux. For example, in Windows, you can run `systeminfo` in the command prompt to see information such as the operating system version, system configuration, and hardware information. In Linux, you can use `uname -a` to display the kernel name, version, and architecture.<|end_of_text|>

In [6]:
%mkdir ../temp
%cd /kaggle/temp

[Errno 2] No such file or directory: '/kaggle/temp'
/content


In [7]:
model.push_to_hub_merged(new_model_name, tokenizer, save_method = "merged_16bit")

Unsloth: You are pushing to hub, but you passed your HF username = suryanshp1.
We shall truncate suryanshp1/Llama-3.1-8B-kali-pentester to Llama-3.1-8B-kali-pentester
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.45 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 34%|███▍      | 11/32 [00:00<00:01, 13.94it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:51<00:00,  1.62s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Llama-3.1-8B-kali-pentester/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Llama-3.1-8B-kali-pentester/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Llama-3.1-8B-kali-pentester/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Llama-3.1-8B-kali-pentester/pytorch_model-00004-of-00004.bin...


  0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/suryanshp1/Llama-3.1-8B-kali-pentester


In [ ]:
model.push_to_hub_gguf(new_model_name, tokenizer, quantization_method = "q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.88 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [02:12<00:00,  4.15s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving suryanshp1/Llama-3.1-8B-kali-pentester/pytorch_model-00001-of-00004.bin...
Unsloth: Saving suryanshp1/Llama-3.1-8B-kali-pentester/pytorch_model-00002-of-00004.bin...
Unsloth: Saving suryanshp1/Llama-3.1-8B-kali-pentester/pytorch_model-00003-of-00004.bin...
